In [ ]:
!pip install langchain huggingface_hub tiktoken Chromadb pypdf sentence-transformers torch accelerate docx2txt
!pip install llama-cpp-python

In [ ]:
# !huggingface-cli download TheBloke/zephyr-7B-alpha-GGUF zephyr-7b-alpha.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
!huggingface-cli download TheBloke/Barcenas-Mistral-7B-GGUF barcenas-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
# !huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False
# !huggingface-cli download TheBloke/Mistral-7B-OpenOrca-oasst_top1_2023-08-25-v1-GGUF mistral-7b-openorca-oasst_top1_2023-08-25-v1.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False



In [17]:
# Definición de la localización del modelo.
MODEL_PATH = "./barcenas-mistral-7b.Q4_K_M.gguf"

In [18]:
import tempfile
import os
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

prompt_template_questions = """Eres un experto en crear preguntas de práctica basadas en material de dominio general. Tu objetivo es escribir preguntas y respuestas a partir de un contexto dado. Lo haces haciendo preguntas sobre el texto a continuación:

{text}
Crea preguntas que prepararán a una persona para responderlas. Asegúrate de no perder ninguna información importante.

PREGUNTAS:"""

refine_template_questions = """ Eres un experto en crear preguntas de práctica basadas en material de estudio. Tu objetivo es ayudar a una persona a prepararse para responder estas preguntas. Hemos recibido algunas preguntas de práctica hasta cierto punto: {existing_answer}. Tenemos la opción de refinar las preguntas existentes o agregar nuevas. (solo si es necesario) con algo más de contexto a continuación.
{text}
Dado el nuevo contexto, refina las preguntas originales en español. Si el contexto no es útil, por favor proporciona las preguntas originales.

PREGUNTAS: """

PROMPT_QUESTIONS = PromptTemplate(template=prompt_template_questions, input_variables=["text"])

REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template_questions,
)


In [20]:
# Se crea el reader para cargar el archivo, en este caso un pdf que debe subir al colab
loader = PyPDFLoader("./example.pdf")
data = loader.load()

In [ ]:
print(data)

In [ ]:
# Combine text from Document into one string for question generation
text_question_gen = ''
for page in data:
   text_question_gen += page.page_content

# Initialize Text Splitter for question generation
# text_splitter_question_gen = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10) # Reduce chunk size
text_splitter_question_gen = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)

# Split text into chunks for question generation
text_chunks_question_gen = text_splitter_question_gen.split_text(text_question_gen)

# Convert chunks into Documents for question generation
docs_question_gen = [Document(page_content=t) for t in text_chunks_question_gen]

llm_question_gen = LlamaCpp(model_path=MODEL_PATH,temperature=0.75,top_p=1,verbose=True,n_ctx=4096)
# llm_question_gen = LlamaCpp(model_path="./barcenas-mistral-7b.Q4_K_M.gguf",temperature=0.1,top_p=1,verbose=True,n_ctx=2048, streaming = True)

# Initialize question generation chain
question_gen_chain = load_summarize_chain(llm=llm_question_gen, chain_type="refine", verbose=True,
                                question_prompt=PROMPT_QUESTIONS, refine_prompt=REFINE_PROMPT_QUESTIONS)

# Run question generation chain
questions = question_gen_chain.run(docs_question_gen)


# Create vector database for answer generation
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})

# Initialize vector store for answer generation
vector_store = Chroma.from_documents(docs_question_gen, embeddings)


llm_answer_gen = LlamaCpp(model_path=MODEL_PATH,temperature=0.75,top_p=1,verbose=True,n_ctx=4096)

# Initialize retrieval chain for answer generation
answer_gen_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, chain_type="stuff",
                                                retriever=vector_store.as_retriever(k=2))

  # Split generated questions into a list of questions
question_list = questions.split("\n")

# Create a directory for storing answers
answers_dir = os.path.join(tempfile.gettempdir(), "answers")
os.makedirs(answers_dir, exist_ok=True)

# Create a single file to save questions and answers
qa_file_path = os.path.join(answers_dir, "questions_and_answers.txt")

with open(qa_file_path, "w") as qa_file:
        # Answer each question and save to the file
        for idx, question in enumerate(question_list):
            answer = answer_gen_chain.run(question)
            qa_file.write(f"Question {idx + 1}: {question}\n")
            qa_file.write(f"Answer {idx + 1}: {answer}\n")
            qa_file.write("--------------------------------------------------\n\n")


# Nota Importante
Debe subir un archivo pdf a colab y pasarselo a este codigo
```python
loader = PyPDFLoader("./example.pdf")
```
para que le funcione correctamente.

# Referencias
[Repositorio de referencia](https://github.com/InsightEdge01/SmartExamApp/tree/master)
